In [1]:
###Cargando los paquetes 
using JuMP, HiGHS, LinearAlgebra, Plots, Random, CSV, DataFrames, XLSX

In [2]:
#Parte d)

In [3]:
#Cargamos los datos 
G_d=XLSX.readtable("DietProblemPolly.xlsx","Data")
### Tipo de G
println("El tipo de G es\n",typeof(G_d),"\n")
### G tiene dos componentes: G.data y G.column_labels
println("G.data es\n",G_d.data,"\n")
println("G.column_labels es\n",G_d.column_labels)

El tipo de G es
XLSX

.DataTable

G.data es


Any[Any[

"Oatmeal", "Chicken ", "Eggs", "Whole milk ", "Cherry pie ", "Pork with beans ", "Minimo "], Any[4.0, 32.0, 13.0, 8.0, 4.0, 14.0, 55.0]

, Any[2.0, 12.0, 54.0, 285.0, 22.0, 80.0, 800.0], Any[3.0, 24.0, 13.0, 9.0, 20.0, 19.0, missing], Any[4.0, 3.0, 2.0, 8.0, 2.0, 2.0, missing]]



G.column_labels es


[:Food, Symbol("Protein "), Symbol("Calcium "), Symbol("Price per seving "), Symbol("Limit per day ")]


In [4]:
data_d = DataFrame(G_d);
display(data_d)

Row,Food,Protein,Calcium,Price per seving,Limit per day
,Any,Any,Any,Any,Any
1,Oatmeal,4.0,2.0,3.0,4.0
2,Chicken,32.0,12.0,24.0,3.0
3,Eggs,13.0,54.0,13.0,2.0
4,Whole milk,8.0,285.0,9.0,8.0
5,Cherry pie,4.0,22.0,20.0,2.0
6,Pork with beans,14.0,80.0,19.0,2.0
7,Minimo,55.0,800.0,missing,missing


In [8]:
###Creando el modelo en conjunto con el solver a usar
modelo_d = Model(HiGHS.Optimizer) 
println(modelo_d)

n=6
m=3

Feasibility
Subject to



3

In [9]:
@variable(modelo_d, x[1:6]>=0,Int) #esto crea el arreglo [x[1] ,...,x[6]]
#agregamos las restricciones de nutrientes
@constraint(modelo_d, protein[j in 2:3], sum(x[i]*data_d[i,j] for i in 1:6) >= data_d[7,j])
#agregamos las restricciones de limite de servings per day
@constraint(modelo_d, limit[i in 1:6],x[i]<=data_d[i,5])

6-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 limit[1] : x[1] <= 4.0
 limit[2] : x[2] <= 3.0
 limit[3] : x[3] <= 2.0
 limit[4] : x[4] <= 8.0
 limit[5] : x[5] <= 2.0
 limit[6] : x[6] <= 2.0

In [10]:
@objective(modelo_d, Min, sum(x[i]*data_d[i,4] for i in 1:6))
println(modelo_d)

Min 3 x[1] + 24 x[2] + 13 x[3] + 9 x[4] + 20 x[5] + 19 x[6]
Subject to


 protein[2] : 4 x[1] + 32 x[2] + 13 x[3] + 8 x[4] + 4 x[5] + 14 x[6] >= 55.0
 protein[3] : 2 x[1] + 12 x[2] + 54 x[3] + 285 x[4] + 22 x[5] + 80 x[6] >= 800.0
 limit[1] : x[1] <= 4.0
 limit[2] : x[2] <= 3.0
 limit[3] : x[3] <= 2.0
 limit[4] : x[4] <= 8.0
 limit[5] : x[5] <= 2.0
 limit[6] : x[6] <= 2.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 x[5] >= 0.0
 x[6] >= 0.0
 x[1] integer
 x[2] integer
 x[3] integer
 x[4] integer
 x[5] integer
 x[6] integer



In [11]:
# Resolver el problema de optimización con HiGHS
optimize!(modelo_d)

println("Optimal value: ", objective_value(modelo_d))

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
2 rows, 6 cols, 12 nonzeros
2 rows, 6 cols, 12 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   2 rows
   6 cols (0 binary, 6 integer, 0 implied int., 0 continuous)
   12 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   18              inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   18              63                71.43%        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      51
  Dual bound        51
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
       

In [12]:
for i in 1:6
    println("El valor de $(x[i]) es ",value(x[i]))
end

El valor de x[1] es 0.0
El valor de x[2] es 1.0
El valor de x[3] es 0.0
El valor de x[4] es 3.0
El valor de x[5] es 0.0
El valor de x[6] es 0.0


In [ ]:
#Parte e

In [13]:
#Cargamos los datos que se encuentran en la otra hoja del excel 
G_e=XLSX.readtable("DietProblemPollyModificado.xlsx","Data parte e")
### Tipo de G
println("El tipo de G_e es\n",typeof(G_e),"\n")
### G tiene dos componentes: G.data y G.column_labels
println("G_e.data es\n",G_e.data,"\n")
println("G_e.column_labels es\n",G_e.column_labels)

El tipo de G_e es
XLSX.DataTable

G_e.data es
Any[Any["Oatmeal", "Chicken ", "Eggs", "Whole milk ", "Cherry pie ", "Pork with beans ", "Minimo "], Any[4.0, 32.0, 13.0, 8.0, 4.0, 14.0, 55.0], Any[2.0, 12.0, 54.0, 285.0, 22.0, 80.0, 800.0], Any[12.0, 0.0, 1.0, 10.0, 39.0, 48.0, 60.0], Any[2.0, 14.0, 11.0, 4.0, 12.0, 3.0, 60.0], Any[2000.0, 6000.0, 5000.0, 1000.0, 10000.0, 3000.0, 

missing], Any[4.0, 3.0, 2.0, 8.0, 2.0, 2.0, missing], Any[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, missing], Any[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, missing], Any[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, missing], Any[0.0, 0.0, 0.0, 0.0, 1.0,

 0.0, missing]]

G_e.column_labels es
[:Food, Symbol("Protein "), Symbol("Calcium "), :Carbohidratos, :Grasas, Symbol("Price per seving "), Symbol("Limit per day "), :Manuel, Symbol("Seba Lillo "), Symbol("Seba Fuenzalida"), Symbol("Alberto ")]


In [14]:
data_e = DataFrame(G_e);
display(data_e)

Row,Food,Protein,Calcium,Carbohidratos,Grasas,Price per seving,Limit per day,Manuel,Seba Lillo,Seba Fuenzalida,Alberto
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,Oatmeal,4.0,2.0,12.0,2.0,2000.0,4.0,0.0,1.0,0.0,0.0
2,Chicken,32.0,12.0,0.0,14.0,6000.0,3.0,1.0,0.0,0.0,0.0
3,Eggs,13.0,54.0,1.0,11.0,5000.0,2.0,0.0,0.0,0.0,0.0
4,Whole milk,8.0,285.0,10.0,4.0,1000.0,8.0,0.0,0.0,0.0,0.0
5,Cherry pie,4.0,22.0,39.0,12.0,10000.0,2.0,0.0,0.0,0.0,1.0
6,Pork with beans,14.0,80.0,48.0,3.0,3000.0,2.0,0.0,0.0,1.0,0.0
7,Minimo,55.0,800.0,60.0,60.0,missing,missing,missing,missing,missing,missing


In [15]:
###Creando el modelo en conjunto con el solver a usar
modelo_e = Model(HiGHS.Optimizer) 
println(modelo_e)

n=6
m=3

Feasibility
Subject to



3

In [16]:
@variable(modelo_e, x[1:6]>=0,Int) #esto crea el arreglo [x[1] ,...,x[6]]
#agregamos las restricciones de nutrientes
@constraint(modelo_e, protein[j in 2:5], sum(x[i]*data_e[i,j] for i in 1:6) >= data_e[7,j])
#agregamos las restricciones de limite de servings per day
@constraint(modelo_e, limit[i in 1:6],x[i]<=data_e[i,7])
#agregamos las restricciones de preferencia de comidas
@constraint(modelo_e, preferencia[j in 8:11], sum(x[i]*data_e[i,j] for i in 1:6)>=1)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 8:11
And data, a 4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 preferencia[8] : x[2] >= 1.0
 preferencia[9] : x[1] >= 1.0
 preferencia[10] : x[6] >= 1.0
 preferencia[11] : x[5] >= 1.0

In [17]:
@objective(modelo_e, Min, sum(x[i]*data_e[i,6] for i in 1:6))
println(modelo_e)

Min 2000 x[1] + 6000 x[2] + 5000 x[3] + 1000 x[4] + 10000 x[5] + 3000 x[6]
Subject to
 protein[2] : 4 x[1] + 32 x[2] + 13 x[3] + 8 x[4] + 4 x[5] + 14 x[6] >= 55.0
 protein[3] : 2 x[1] + 12 x[2] + 54 x[3] + 285 x[4] + 22 x[5] + 80 x[6] >= 800.0
 protein[4] : 12 x[1] + x[3] + 10 x[4] + 39 x[5] + 48 x[6] >= 60.0
 protein[5] : 2 x[1] + 14 x[2] + 11 x[3] + 4 x[4] + 12 x[5] + 3 x[6] >= 60.0
 preferencia[8] : x[2] >= 1.0
 preferencia[9] : x[1] >= 1.0
 preferencia[10] : x[6] >= 1.0
 preferencia[11] : x[5] >= 1.0
 limit[1] : x[1] <= 4.0
 limit[2] : x[2] <= 3.0
 limit[3] : x[3] <= 2.0
 limit[4] : x[4] <= 8.0
 limit[5] : x[5] <= 2.0
 limit[6] : x[6] <= 2.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 x[5] >= 0.0
 x[6] >= 0.0
 

x[1] integer
 x[2] integer
 x[3] integer
 x[4] integer
 x[5] integer
 x[6] integer



In [18]:
# Resolver el problema de optimización con HiGHS
optimize!(modelo_e)

println("Optimal value: ", objective_value(modelo_e))

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
4 rows, 6 cols, 23 nonzeros
2 rows, 6 cols, 12 nonzeros
Objective function is integral with scale 0.001

Solving MIP model with:
   2 rows
   6 cols (2 binary, 4 integer, 0 implied int., 0 continuous)
   12 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   23000           inf                  inf        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      29000
  Dual bound        29000
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    29000 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
          

In [19]:
for i in 1:6
    println("El valor de $(x[i]) es ",value(x[i]))
end

El valor de x[1] es 1.0
El valor de x[2] es 1.0
El valor de x[3] es 0.0
El valor de x[4] es 8.0
El valor de x[5] es 1.0
El valor de x[6] es 1.0


In [ ]:
#Parte f

In [20]:
#Cargamos los datos 
G_f=XLSX.readtable("DietProblemPolly.xlsx","Data")
### Tipo de G_f
println("El tipo de G_f es\n",typeof(G_f),"\n")
### G_f tiene dos componentes: G_f.data y G_f.column_labels
println("G_f.data es\n",G_f.data,"\n")
println("G_f.column_labels es\n",G_f.column_labels)

El tipo de G_f es
XLSX.DataTable

G_f.data es
Any[Any["Oatmeal", "Chicken ", "Eggs", "Whole milk ", "Cherry pie ", "Pork with beans ", "Minimo "], Any[4.0, 32.0, 13.0, 8.0, 4.0, 14.0, 55.0], Any[2.0, 12.0, 54.0, 285.0, 22.0, 80.0, 800.0], Any[3.0, 24.0, 13.0, 9.0, 20.0, 19.0, missing], Any[

4.0, 3.0, 2.0, 8.0, 2.0, 2.0, missing]]

G_f.column_labels es
[:Food, Symbol("Protein "), Symbol("Calcium "), Symbol("Price per seving "), Symbol("Limit per day ")]


In [21]:
data_f = DataFrame(G_f);
display(data_f)

Row,Food,Protein,Calcium,Price per seving,Limit per day
,Any,Any,Any,Any,Any
1,Oatmeal,4.0,2.0,3.0,4.0
2,Chicken,32.0,12.0,24.0,3.0
3,Eggs,13.0,54.0,13.0,2.0
4,Whole milk,8.0,285.0,9.0,8.0
5,Cherry pie,4.0,22.0,20.0,2.0
6,Pork with beans,14.0,80.0,19.0,2.0
7,Minimo,55.0,800.0,missing,missing


In [22]:
###Creando el modelo en conjunto con el solver a usar
modelo_f = Model(HiGHS.Optimizer) 
println(modelo_f)

n=6
m=3

Feasibility
Subject to



3

In [23]:
@variable(modelo_f, x[1:6]>=0,Int) #esto crea el arreglo [x[1] ,...,x[6]]
#agregamos las restricciones de nutrientes
@constraint(modelo_f, protein[j in 2:3], sum(x[i]*data_f[i,j] for i in 1:6) >= data_f[7,j])
#agregamos las restricciones de limite de servings per day
@constraint(modelo_f, limit[i in 1:6],x[i]<=data_f[i,5])

6-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 limit[1] : x[1] <= 4.0
 limit[2] : x[2] <= 3.0
 limit[3] : x[3] <= 2.0
 limit[4] : x[4] <= 8.0
 limit[5] : x[5] <= 2.0
 limit[6] : x[6] <= 2.0

In [24]:
@objective(modelo_f, Min, sum(x[i]*data_f[i,4] for i in 1:6))
println(modelo_f)

Min 3 x[1] + 24 x[2] + 13 x[3] + 9 x[4] + 20 x[5] + 19 x[6]
Subject to
 protein[2] : 4 x[1] + 32 x[2] + 13 x[3] + 8 x[4] + 4 x[5] + 14 x[6] >= 55.0
 protein[3] : 2 x[1] + 12 x[2] + 54 x[3] + 285 x[4] + 22 x[5] + 80 x[6] >= 800.0
 limit[1] : x[1] <= 4.0
 limit[2] : x[2] <= 3.0
 limit[3] : x[3] <= 2.0
 limit[4] : x[4] <= 8.0
 limit[5] : x[5] <= 2.0
 limit[6] : x[6] <= 2.0
 x[1] >= 0.0


 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 x[5] >= 0.0
 x[6] >= 0.0
 x[1] integer
 x[2] integer
 x[3] integer
 x[4] integer
 x[5] integer
 x[6] integer



In [25]:
# Resolver el problema de optimización con HiGHS
optimize!(modelo_f)

println("Optimal value: ", objective_value(modelo_f))

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
2 rows, 6 cols, 12 nonzeros
2 rows, 6 cols, 12 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   2 rows
   6 cols (0 binary, 6 integer, 0 implied int., 0 continuous)
   12 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   18              inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   18              63                71.43%        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      51
  Dual bound        51
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
       

In [26]:
for i in 1:6
    println("El valor de $(x[i]) es ",value(x[i]))
end

El valor de x[1] es 0.0
El valor de x[2] es 1.0
El valor de x[3] es 0.0
El valor de x[4] es 3.0
El valor de x[5] es 0.0
El valor de x[6] es 0.0


In [ ]:
#Parte g)

In [27]:
#Cargamos los datos 
G_g=XLSX.readtable("DietProblemPolly.xlsx","Data")
### Tipo de G_g
println("El tipo de G_g es\n",typeof(G_g),"\n")
### G_g tiene dos componentes: G_g.data y G_g.column_labels
println("G_g.data es\n",G_g.data,"\n")
println("G_g.column_labels es\n",G_g.column_labels)

El tipo de G_g es
XLSX.DataTable

G_g.data es
Any[Any["Oatmeal", "Chicken ", "Eggs", "Whole milk ", "Cherry pie ", "Pork with beans ", "Minimo "], Any[4.0, 32.0, 13.0, 

8.0, 4.0, 14.0, 55.0], Any[2.0, 12.0, 54.0, 285.0, 22.0, 80.0, 800.0], Any[3.0, 24.0, 13.0, 9.0, 20.0, 19.0, missing], Any[4.0, 3.0, 2.0, 8.0, 2.0, 2.0, missing]]

G_g.column_labels es
[:Food, Symbol("Protein "), Symbol("Calcium "), Symbol("Price per seving "), Symbol("Limit per day ")]


In [28]:
data_g = DataFrame(G_g);
display(data_g)

Row,Food,Protein,Calcium,Price per seving,Limit per day
,Any,Any,Any,Any,Any
1,Oatmeal,4.0,2.0,3.0,4.0
2,Chicken,32.0,12.0,24.0,3.0
3,Eggs,13.0,54.0,13.0,2.0
4,Whole milk,8.0,285.0,9.0,8.0
5,Cherry pie,4.0,22.0,20.0,2.0
6,Pork with beans,14.0,80.0,19.0,2.0
7,Minimo,55.0,800.0,missing,missing


In [29]:
###Creando el modelo en conjunto con el solver a usar
modelo_g = Model(HiGHS.Optimizer) 
println(modelo_g)

n=6
m=3
B=3

Feasibility
Subject to

3

In [30]:
#creamos variables para ver cuantas porciones de cada comida son
@variable(modelo_g, x[1:6]>=0,Int) #esto crea el arreglo [x[1] ,...,x[6]]
#creamos unas variables binarias para ver si la comida i se come o no 
@variable(modelo_g, y[1:6], Bin) 
#agregamos las restricciones de nutrientes
@constraint(modelo_g, protein[j in 2:3], sum(x[i]*data_g[i,j] for i in 1:6) >= data_g[7,j])
#agregamos las restricciones de limite de servings per day
@constraint(modelo_g, limit[i in 1:6],x[i]<=data_g[i,5])
#restriccion big-M para ver si comida i se come o no 
@constraint(modelo_g, Secome[i in 1:6],x[i]>=0-B*(1-y[i]))
#restriccion big-M para ver si comida i se come o no 
@constraint(modelo_g, NoSeCome[i in 1:6],x[i]<=B*y[i])
#restriccion de que no se pueden comprar mas de B comidas 
@constraint(modelo_g, limiteComidas,sum(y[i] for i in 1:6)<=B)

limiteComidas : y[1] + y[2] + y[3] + y[4] + y[5] + y[6] <= 3.0

In [31]:
@objective(modelo_g, Min, sum(x[i]*data_g[i,4] for i in 1:6))
println(modelo_g)

Min 3 x[1] + 24 x[2] + 13 x[3] + 9 x[4] + 20 x[5] + 19 x[6]
Subject to


 protein[2] : 4 x[1] + 32 x[2] + 13 x[3] + 8 x[4] + 4 x[5] + 14 x[6] >= 55.0
 protein[3] : 2 x[1] + 12 x[2] + 54 x[3] + 285 x[4] + 22 x[5] + 80 x[6] >= 800.0
 Secome[1] : x[1] - 3 y[1] >= -3.0
 Secome[2] : x[2] - 3 y[2] >= -3.0
 Secome[3] : x[3] - 3 y[3] >= -3.0
 Secome[4] : x[4] - 3 y[4] >= -3.0
 Secome[5] : x[5] - 3 y[5] >= -3.0
 Secome[6] : x[6] - 3 y[6] >= -3.0
 limit[1] : x[1] <= 4.0
 limit[2] : x[2] <= 3.0
 limit[3] : x[3] <= 2.0
 limit[4] : x[4] <= 8.0
 limit[5] : x[5] <= 2.0
 limit[6] : x[6] <= 2.0
 NoSeCome[1] : x[1] - 3 y[1] <= 0.0
 NoSeCome[2] : x[2] - 3 y[2] <= 0.0
 NoSeCome[3] : x[3] - 3 y[3] <= 0.0
 NoSeCome[4] : x[4] - 3 y[4] <= 0.0
 NoSeCome[5] : x[5] - 3 y[5] <= 0.0
 NoSeCome[6] : x[6] - 3 y[6] <= 0.0
 limiteComidas : y[1] + y[2] + y[3] + y[4] + y[5] + y[6] <= 3.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 x[5] >= 0.0
 x[6] >= 0.0
 x[1] integer
 x[2] integer
 x[3] integer
 x[4] integer
 x[5] integer
 x[6] integer
 y[1] binary
 y[2] binary
 y[3] binary
 

y[4] binary
 y[5] binary
 y[6] binary



In [32]:
# Resolver el problema de optimización con HiGHS
optimize!(modelo_g)

println("Optimal value: ", objective_value(modelo_g))

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
9 rows, 12 cols, 30 nonzeros
8 rows, 11 cols, 27 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   8 rows
   11 cols (6 binary, 5 integer, 0 implied int., 0 continuous)
   27 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   18              inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   50.07654185     60                16.54%        0      0      0         4     0.0s

18.2% inactive integer columns, restarting
Model after restart has 6 rows, 8 cols (6 bin., 2 int., 0 impl., 0 cont.), and 16 nonzeros

         0       0         0 

In [33]:
for i in 1:6
    println("El valor de $(x[i]) es ",value(x[i]))

end

El valor de x[1] es -0.0
El valor de x[2] es 1.0
El valor de x[3] es 0.0
El valor de x[4] es 

3.0
El valor de x[5] es 0.0
El valor de x[6] es 0.0


In [34]:
for i in 1:6
    println("El valor de $(y[i]) es ",value(y[i]))

end

El valor de y[1] es -0.0
El valor de y[2] es 1.0
El valor de y[3] es -0.0
El valor de y[4] es 1.0
El valor de y[5] es 0.0
El valor de y[6] es 0.0


In [ ]:
#Parte h)

In [36]:
#Cargamos los datos 
G_h=XLSX.readtable("DietProblemPolly.xlsx","Data")
### Tipo de G_h
println("El tipo de G_h es\n",typeof(G_h),"\n")
### G_h tiene dos componentes: G_h.data y G_h.column_labels
println("G_h.data es\n",G_h.data,"\n")
println("G_h.column_labels es\n",G_h.column_labels)

El tipo de G_h es
XLSX.DataTable

G_h.data es
Any[Any["Oatmeal", "Chicken ", "Eggs", "Whole milk ", "Cherry pie ", "Pork with beans ", "Minimo "], Any[4.0, 32.0, 13.0, 8.0, 4.0, 14.0, 55.0], Any[2.0, 12.0, 54.0, 285.0, 22.0, 80.0, 800.0], Any[3.0, 24.0, 13.0, 9.0, 20.0, 19.0, missing], Any[

4.0, 3.0, 2.0, 8.0, 2.0, 2.0, missing]]

G_h.column_labels es
[:Food, Symbol("Protein "), Symbol("Calcium "), Symbol("Price per seving "), Symbol("Limit per day ")]


In [37]:
data_h = DataFrame(G_h);
display(data_h)

Row,Food,Protein,Calcium,Price per seving,Limit per day
,Any,Any,Any,Any,Any
1,Oatmeal,4.0,2.0,3.0,4.0
2,Chicken,32.0,12.0,24.0,3.0
3,Eggs,13.0,54.0,13.0,2.0
4,Whole milk,8.0,285.0,9.0,8.0
5,Cherry pie,4.0,22.0,20.0,2.0
6,Pork with beans,14.0,80.0,19.0,2.0
7,Minimo,55.0,800.0,missing,missing


In [38]:
###Creando el modelo en conjunto con el solver a usar
modelo_h = Model(HiGHS.Optimizer) 
println(modelo_h)

n=6
m=3

Feasibility
Subject to



3

In [39]:
#variables de decision
@variable(modelo_h, x[1:6]>=0,Int) #esto crea el arreglo [x[1] ,...,x[6]]
#variable z binaria
@variable(modelo_h, z, Bin) 
#agregamos las restricciones de nutrientes
@constraint(modelo_h, protein[j in 2:3], sum(x[i]*data_h[i,j] for i in 1:6) >= data_h[7,j])
#agregamos las restricciones de limite de servings per day
@constraint(modelo_h, limit[i in 1:6],x[i]<=data_h[i,5])
#agregamos restrccion para pork with beans
@constraint(modelo_h, PorkWithBeans, x[6]<=data_h[6,5]*(1-z))
#agregamos restrccion para cherry pie
@constraint(modelo_h, CherryPie, x[5]<=data_h[5,5]*z)

CherryPie : x[5] - 2 z <= 0.0

In [40]:
@objective(modelo_h, Min, sum(x[i]*data_h[i,4] for i in 1:6))
println(modelo_h)

Min 3 x[1] + 24 x[2] + 13 x[3] + 9 x[4] + 20 x[5] + 19 x[6]


Subject to
 protein[2] : 4 x[1] + 32 x[2] + 13 x[3] + 8 x[4] + 4 x[5] + 14 x[6] >= 55.0
 protein[3] : 2 x[1] + 12 x[2] + 54 x[3] + 285 x[4] + 22 x[5] + 80 x[6] >= 800.0
 limit[1] : x[1] <= 4.0
 limit[2] : x[2] <= 3.0
 limit[3] : x[3] <= 2.0
 limit[4] : x[4] <= 8.0
 limit[5] : x[5] <= 2.0
 limit[6] : x[6] <= 2.0
 PorkWithBeans : x[6] + 2 z <= 2.0
 CherryPie : x[5] - 2 z <= 0.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 x[5] >= 0.0
 x[6] >= 0.0
 x[1] integer
 x[2] integer
 x[3] integer
 x[4] integer
 x[5] integer
 x[6] integer
 z binary



In [41]:
# Resolver el problema de optimización con HiGHS
optimize!(modelo_h)

println("Optimal value: ", objective_value(modelo_h))

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
4 rows, 7 cols, 16 nonzeros
4 rows, 7 cols, 16 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   4 rows
   7 cols (1 binary, 6 integer, 0 implied int., 0 continuous)
   16 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   18              inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   18              63                71.43%        0      0      0         0     0.0s
 R       0       0         0   0.00%   50.25           60                16.25%        0      0      0         3     0.0s

14.3% inactive integer columns, restarting
Mo

In [42]:
for i in 1:6
    println("El valor de $(x[i]) es ",value(x[i]))
end

El valor de x[1] es 0.0
El valor de x[2] es 1.0
El valor de x[3] es 0.0
El valor de x[4] es 3.0
El valor de x[5] es 0.0
El valor de x[6] es 0.0


In [ ]:
#Parte i)

In [43]:
#Cargamos los datos 
G_i=XLSX.readtable("DietProblemPolly.xlsx","Data")
### Tipo de G_i
println("El tipo de G_i es\n",typeof(G_i),"\n")
### G_i tiene dos componentes: G_i.data y G_i.column_labels
println("G_i.data es\n",G_i.data,"\n")
println("G_i.column_labels es\n",G_i.column_labels)

El tipo de G_i es
XLSX.DataTable

G_i.data es
Any[Any["Oatmeal", "Chicken ", "Eggs", "Whole milk ", "Cherry pie ", "Pork with beans ", "Minimo "], Any[4.0, 32.0, 13.0, 8.0, 4.0, 14.0, 55.0], Any[2.0, 12.0, 54.0, 285.0, 22.0, 80.0, 800.0], Any[3.0, 24.0, 13.0, 9.0, 20.0, 19.0, missing], Any[4.0, 3.0, 2.0, 8.0, 2.0, 2.0, missing]]

G_i.column_labels es
[:Food, Symbol("Protein "), Symbol("Calcium "), Symbol("Price per seving "), Symbol("Limit per day ")]

In [44]:
data_i = DataFrame(G_i);
display(data_i)

Row,Food,Protein,Calcium,Price per seving,Limit per day
,Any,Any,Any,Any,Any
1,Oatmeal,4.0,2.0,3.0,4.0
2,Chicken,32.0,12.0,24.0,3.0
3,Eggs,13.0,54.0,13.0,2.0
4,Whole milk,8.0,285.0,9.0,8.0
5,Cherry pie,4.0,22.0,20.0,2.0
6,Pork with beans,14.0,80.0,19.0,2.0
7,Minimo,55.0,800.0,missing,missing


In [45]:
###Creando el modelo en conjunto con el solver a usar
modelo_i = Model(HiGHS.Optimizer) 
println(modelo_i)

n=6
m=2
T=1

Feasibility
Subject to



1

In [46]:
#variables de decision en que comidas comprar y cantidad
@variable(modelo_i, x[1:6]>=0,Int) #esto crea el arreglo [x[1] ,...,x[6]]
#variable binaria z
@variable(modelo_i, z[1:2],Bin)
#agregamos las restricciones de nutrientes por la variable z que me dice si se debe cumplir o no ese requerimiento
@constraint(modelo_i, protein[j in 2:3], sum(x[i]*data_i[i,j] for i in 1:6) >= data_i[7,j]*z[j-1])
#agregamos las restricciones de limite de servings per day
@constraint(modelo_i, limit[i in 1:6],x[i]<=data_i[i,5])
#agregamos la restriccion de que al menos T requerimientos se deben cumplir
@constraint(modelo_i, Trequerimientos, sum(z[k] for k in 1:2)>=T)

Trequerimientos : z[1] + z[2] >= 1.0

In [47]:
@objective(modelo_i, Min, sum(x[i]*data_i[i,4] for i in 1:6))
println(modelo_i)

Min 3 x[1] + 24 x[2] + 13 x[3] + 9 x[4] + 20 x[5] + 19 x[6]
Subject to
 protein[2] : 4 x[1] + 32 x[2] + 13 x[3] + 8 x[4] + 4 x[5] + 14 x[6] - 55 z[1] >= 0.0
 protein[3] : 2 x[1] + 12 x[2] + 54 x[3] + 285 x[4] + 22 x[5] + 80 x[6] - 800 z[2] >= 0.0
 Trequerimientos : z[1] + z[2] >= 1.0
 limit[1] : x[1] <= 4.0
 limit[2] : x[2] <= 3.0
 limit[3] : x[3] <= 2.0
 limit[4] : x[4] <= 8.0
 

limit[5] : x[5] <= 2.0
 limit[6] : x[6] <= 2.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 x[5] >= 0.0
 x[6] >= 0.0
 x[1] integer
 x[2] integer
 x[3] integer
 x[4] integer
 x[5] integer
 x[6] integer
 z[1] binary
 z[2] binary



In [48]:
# Resolver el problema de optimización con HiGHS
optimize!(modelo_i)

println("Optimal value: ", objective_value(modelo_i))

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
3 rows, 8 cols, 16 nonzeros
3 rows, 8 cols, 16 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   3 rows
   8 cols (2 binary, 6 integer, 0 implied int., 0 continuous)
   16 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      27
  Dual bound        27
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    27 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (

In [49]:
for i in 1:6
    println("El valor de $(x[i]) es ",value(x[i]))
end

El valor de x[1] es 0.0
El valor de x[2] es 0.0
El valor de x[3] es 0.0
El valor de x[4] es 3.0
El valor de x[5] es 0.0
El valor de x[6] es 0.0
